# 向量化字符串操作

使用 `Python` 的一个优势就是字符串处理起来比较容易。  
在此基础上创建的 `Pandas` 同样提供了一系列向量化字符串操作（`vectorized string operation`），它们都是在处理（清洗）现实工作中的数据时不可或缺的功能。

In [1]:
import numpy as np
import pandas as pd

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
  img {width:30%; height: 30%;}
</style>

## 1. Pandas字符串操作简介

In [3]:
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

**向量化**操作简化了纯数值的数组操作语法——我们不需要再担心数组的长度或维度，只需要关心需要的操作。  
然而，由于 `NumPy` 并没有为字符串数组提供简单的接口，因此需要通过繁琐的 `for` 循环来解决问题：

In [4]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

In [5]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
[s.capitalize() for s in data]  # 假如数据中出现了缺失值，那么这样做就会引起异常

AttributeError: 'NoneType' object has no attribute 'capitalize'

`Pandas` 为包含字符串的 `Series` 和 `Index` 对象提供的 `str` 属性堪称两全其美的方法，它既可以满足向量化字符串操作的需求，又可以正确地处理缺失值。

In [6]:
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

In [7]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

## 2. Pandas字符串方法列表

### 2.1. 与Python字符串方法相似的方法

几乎所有 `Python` 内置的字符串方法都被复制到 `Pandas` 的向量化字符串方法中。  
下面的表格列举了 `Pandas` 的 `str` 方法借鉴 `Python` 字符串方法的内容：

| -- | -- | -- | -- |
| -- | -- | -- | -- |
| len() | lower() | translate() | islower() |
| ljust() | upper() | startswith() | isupper() |
| rjust() | find() | endswith() | isnumeric() |
| center() | rfind() | isalnum() | isdecimal() |
| zfill() | index() | isalpha() | split() |
| strip() | rindex() | isdigit() | rsplit() |
| rstrip() | capitalize() | isspace() | partition() |
| lstrip() | swapcase() | istitle() | rpartition() |

In [8]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

In [9]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

In [10]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

In [11]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

### 2.2. 使用正则表达式的方法

**Pandas向量化字符串方法与Python标准库的re模块函数的对应关系**

|方法 | 描述 |
| -- | -- |
|match() | 对每个元素调用 re.match()，返回布尔类型值 |
|extract() | 对每个元素调用 re.match()，返回匹配的字符串组（groups） |
|findall() | 对每个元素调用 re.findall() |
|replace() | 用正则模式替换字符串 |
|contains() | 对每个元素调用 re.search()，返回布尔类型值 |
|count() | 计算符合正则模式的字符串的数量 |
|split() | 等价于 str.split()，支持正则表达式 |
|rsplit() | 等价于 str.rsplit()，支持正则表达式 |

In [12]:
monte.str.extract('([A-Za-z]+)')  # 可以提取元素前面的连续字母作为每个人的名字（first name）

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


In [13]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')  # 找出所有开头和结尾都是辅音字母的名字

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

### 2.3. 其他字符串方法

**其他Pandas字符串方法**

| 方法 | 描述 |
| -- | -- |
| get() | 获取元素索引位置上的值，索引从 0 开始 |
| slice() | 对元素进行切片取值 |
| slice_replace() | 对元素进行切片替换 |
| cat() | 连接字符串（此功能比较复杂，建议阅读文档） |
| repeat() | 重复元素 |
| normalize() | 将字符串转换为 Unicode 规范形式 |
| pad() | 在字符串的左边、右边或两边增加空格 |
| wrap() | 将字符串按照指定的宽度换行 |
| join() | 用分隔符连接 Series 的每个元素 |
| get_dummies() | 按照分隔符提取每个元素的 dummy 变量，转换为独热（onehot）编码的 DataFrame |

#### 2.3.1. 向量化字符串的取值与切片操作

这里需要特别指出的是，`get()` 与 `slice()` 操作可以从每个字符串数组中获取向量化元素。

In [16]:
monte.str[0:3]  # <==> monte.str.slice(0,3)

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

`df.str.get(i)` 与 `df.str[i]` 的按索引取值效果类似。  
`get()` 与 `slice()` 操作还可以在 `split()` 操作之后使用。

In [17]:
monte.str.split().str.get(-1)  # 获取每个姓名的姓（last name），可以结合使用 split() 与 get()

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

#### 2.3.2. 指标变量

另一个需要多花点儿时间解释的是 `get_dummies()` 方法。当你的数据有一列包含了若干已被编码的指标（`coded indicator`）时，这个方法就能派上用场了。

In [18]:
# 例如，假设有一个包含了某种编码信息的数据集，如 A= 出生在美国、B= 出生在英国、C= 喜欢奶酪、D= 喜欢午餐肉：
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C', 'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


In [19]:
full_monte['info'].str.get_dummies('|')  # get_dummies() 方法可以让你快速将这些指标变量分割成一个独热编码的 DataFrame（每个元素都是 0 或 1）

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


## 3. 案例：食谱数据库

In [21]:
!cp ../data/recipeitems-latest.json.bz2 /tmp
!bzip2 -d /tmp/recipeitems-latest.json.bz2

In [22]:
try:
    recipes = pd.read_json('/tmp/recipeitems-latest.json')
except ValueError as e:
    print("ValueError:", e)  # 数据里有“trailing data”（数据断行）的 ValueError 错误。从网上搜索这个错误，得知原因好像是虽然文件中的每一行都是一个有效的 JSON 对象，但是全文却不是这样。

ValueError: Trailing data


In [35]:
import io

with open('/tmp/recipeitems-latest.json') as f:
    line = f.readline()
pd.read_json(io.StringIO(line)).shape  # pd.read_json(line).shape会报错，据说pandas 1.1 开始pd.read_json()不再能读取简单的字符串

(2, 12)

显然每一行都是一个有效的 `JSON` 对象，因此需要将这些字符串连接在一起。  
解决这个问题的一种方法就是新建一个字符串，将所有行 `JSON` 对象连接起来，然后再通过 `pd.read_json` 来读取所有数据：

In [38]:
# 将文件内容读取成Python数组
with open('/tmp/recipeitems-latest.json', 'r') as f:
    data = (line.strip() for line in f)         # 提取每一行内容
    data_json = "[{0}]".format(','.join(data))  # 将所有内容合并成一个列表

recipes = pd.read_json(io.StringIO(data_json))  # 用JSON形式读取数据

MemoryError: 

### 3.1. 制作简易的美食推荐系统

### 3.2. 继续完善美食推荐系统

In [ ]:
!rm -f /tmp/recipeitems-latest.json.bz2